#### 1. Модифицировать приложение из предыдущего домашнего задания:
- приложение должно собирать вакансии сразу с двух сайтов hh.ru и superjob.ru 
- собранные данные должны быть приведены к общей структуре
- разделить зарплату на две составляющие (мин. и макс.) и сохранить в виде int. Если валюта указана другая, привести все к рублям. 

In [1]:
import pandas as pd
import requests
import json
import re
import hashlib
from bs4 import BeautifulSoup as bs
from pprint import pprint

In [2]:
def get_html(link, headers):
    with requests.session() as s:
        r = s.get(link, headers=headers)
    if r.status_code == 200:
        return r.text
    else:
        return False

In [3]:
def get_hh_vacancies(search_queries=[], page_count=1, headers={'User-agent': 'Mozilla/5.0'}):

    def get_salary_info(link, headers):
        html = get_html(link, headers)
        _min_salary, _max_salary = 0, 0
        if html:
            soup = bs(html, 'html.parser')
            tags = soup.find_all('p', class_='vacancy-salary')
            _salary = re.findall(r'\d+', re.sub(r'\s', '', bs(str(tags[0])).get_text()))
            if len(_salary) == 0:
                pass
            elif len(_salary) == 1:
                _min_salary = _max_salary = _salary[0]
            elif len(_salary) == 2:
                _min_salary,  _max_salary = _salary
            else:
                pass
            return _min_salary, _max_salary
        else:
            return _min_salary, _max_salary

    def get_vacancy_info(search_queries, page_count, headers):
        if len(search_queries) > 0:
            _df_list = []
            for search_query in search_queries:
                text = '+'.join(search_query.split(' ')).capitalize()
                for i in range(page_count):
                    link = f'https://hh.ru/search/vacancy?area=1&text={text}&page={i}'
                    html = get_html(link, headers)
                    if html:
                        soup = bs(html, 'html.parser')
                        tags = soup.find_all('a', class_='bloko-link HH-LinkModifier')
                        _vacancies = {
                            'id': [],
                            'name': [],
                            'min_salary': [],
                            'max_salary': [],
                            'link': []
                        }
                        for tag in tags:
                            _id = 'hh' + re.findall(r'(\d+)\??', tag['href'])[0]
                            # _name = bs(str(tag)).get_text()
                            _name = tag.get_text()
                            _min_salary, _max_salary = get_salary_info(tag['href'], headers)
                            _link = tag['href']
                            _vacancies['id'].append(hashlib.sha256(_id.encode('utf-8')).hexdigest())
                            _vacancies['name'].append(_name)
                            _vacancies['min_salary'].append(_min_salary)
                            _vacancies['max_salary'].append(_max_salary)
                            _vacancies['link'].append(_link)
                        _df_temp = pd.DataFrame(_vacancies)
                    else:
                        return False
                    _df_list.append(_df_temp)
            return pd.concat(_df_list).drop_duplicates(subset='id').reset_index(drop=True)
        else:
            return False
        
    return get_vacancy_info(search_queries, page_count, headers)


In [10]:
def get_superjob_vacancies(search_queries=[], page_count=1, headers={'User-agent': 'Mozilla/5.0'}):

    def get_salary_info(link, headers):
        html = get_html(link, headers)
        _min_salary, _max_salary = 0, 0
        html = get_html(link, headers)
        if html:
            soup = bs(html, 'html.parser')
            tags = soup.find_all('meta', property="og:description")
            _salary = re.findall(r'\d+', re.findall(r'зарплата.+\.', re.sub(r'\s', '', tags[0]['content']))[0])
            if len(_salary) == 0:
                pass
            elif len(_salary) == 1:
                _min_salary = _max_salary = _salary[0]
            elif len(_salary) == 2:
                _min_salary,  _max_salary = _salary
            else:
                pass
            return _min_salary, _max_salary
        else:
            return _min_salary, _max_salary

    def get_vacancy_info(search_queries, page_count, headers):
        if len(search_queries) > 0:
            _df_list = []
            for search_query in search_queries:
                text = '%20'.join(search_query.split(' ')).capitalize()
                for i in range(1, page_count+1):
                    link = f'https://www.superjob.ru/vacancy/search/?geo%5Bt%5D%5B0%5D=4&keywords={text}&page={i}'
                    html = get_html(link, headers)
                    if html:
                        soup = bs(html, 'html.parser')
                        tags = soup.find_all('a', href=True)
                        _vacancies = {
                            'id': [],
                            'name': [],
                            'min_salary': [],
                            'max_salary': [],
                            'link': []
                        }
                        for tag in tags:
                            if len(re.findall(r'<a class="icMQ_ _1QIBo f-test-link-.+', str(tag))) > 0:
                                _id = 'sj' + re.findall(r'(\d+)\.', tag['href'])[0]
                                _name = tag.get_text()
                                _min_salary, _max_salary = get_salary_info('https://www.superjob.ru' + tag['href'], headers)
                                _link = 'https://www.superjob.ru' + tag['href']
                                _vacancies['id'].append(hashlib.sha256(_id.encode('utf-8')).hexdigest())
                                _vacancies['name'].append(_name)
                                _vacancies['min_salary'].append(_min_salary)
                                _vacancies['max_salary'].append(_max_salary)
                                _vacancies['link'].append(_link)
                        _df_temp = pd.DataFrame(_vacancies)
                    else:
                        return False
                    _df_list.append(_df_temp)
            return pd.concat(_df_list).drop_duplicates(subset='id').reset_index(drop=True)
        else:
            return False

    return get_vacancy_info(search_queries, page_count, headers)


In [11]:
def get_vacancies(search_queries=[], page_count=1, headers={'User-agent': 'Mozilla/5.0'}):

    try:
        
        hh = get_hh_vacancies(search_queries, page_count, headers)
        superjob = get_superjob_vacancies(search_queries, page_count, headers)

        if isinstance(hh, pd.DataFrame) and isinstance(superjob, pd.DataFrame):
            hh['min_salary'] = hh['min_salary'].astype(int)
            hh['max_salary'] = hh['max_salary'].astype(int)
            hh['source'] = 'hh'
            superjob['min_salary'] = superjob['min_salary'].astype(int)
            superjob['max_salary'] = superjob['max_salary'].astype(int)
            superjob['source'] = 'superjob'
            return pd.concat([hh, superjob])
        else:
            return 'Something go wrong..'
        
    except:
        
        return 'Something go wrong..'

In [12]:
search_queries = ['программист', 'разработчик']
df = get_vacancies(search_queries, page_count=1)

In [13]:
df.head()

,id,name,min_salary,max_salary,link,source
0,4dfb9a2a504562f1ae9a44d599b8e961f483b8045c0264...,Full-Stack Разработчик JavaScript (Node.js + A...,120000,120000,https://hh.ru/vacancy/32859767?query=%D0%9F%D1...,hh
1,e909bfa7060003ed025147907b66872f84c17cc9047175...,Web-программист/PHP-программист,0,0,https://hh.ru/vacancy/32728996?query=%D0%9F%D1...,hh
2,fff950fe5cc0942dfc5d4780a800e49637d450deb6d562...,Программист C/С++,80000,120000,https://hh.ru/vacancy/32666256?query=%D0%9F%D1...,hh
3,519f828a46f16331e48a5cf9090d71d646b55223553f6b...,Программист (C#\ ASP.NET\MVC),100000,100000,https://hh.ru/vacancy/33036570?query=%D0%9F%D1...,hh
4,7928c9bda1b8f2081020d37c817bd776bab6b5a2262715...,Middle PHP-разработчик/Программист (Bitrix),100000,120000,https://hh.ru/vacancy/31746332?query=%D0%9F%D1...,hh


#### 2. Реализовать сохранение полученных вакансий в СУБД (на выбор SQLite или MongoDB)

In [16]:
from pymongo import MongoClient

In [17]:
def save_to_db(data_frame, db_name):
    try:
        
        client = MongoClient('mongodb://127.0.0.1:27017')
        db = client[db_name]
        
        data = data_frame.to_dict('records')
        
        for item in data:
            objects = db.vacancydb.find({'id': item['id']})
            if len(list(objects)) == 0:
                db.vacancydb.insert_one(item)
        
        return f'Data saved to: {db_name}'
        
    except:
    
        return 'Something go wrong..'

In [21]:
db_name = 'vacancydb'

client = MongoClient('mongodb://127.0.0.1:27017')
db = client[db_name]
db.vacancydb.create_index('id', unique=True)

'id_1'

In [22]:
save_to_db(data_frame=df, db_name=db_name)

'Data saved to: vacancydb'

#### 3.Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введенной суммы.

In [26]:
def search_in_db(db_name, min_value):
    client = MongoClient('mongodb://127.0.0.1:27017')
    db = client[db_name]
    objects = db.vacancydb.find({'min_salary': {'$gt': min_value}})
    return list(objects)

In [30]:
search_in_db(db_name='vacancydb', min_value=180000)

[{'_id': ObjectId('5d5afddd30c104b5b6a3f87c'),
  'id': 'ceeaf3d19841cf41bcbd8974232e78b44d194e73726f222805b2424676d5ed47',
  'name': 'Senior Front-end Developer (JavaScript, React)',
  'min_salary': 200000,
  'max_salary': 200000,
  'link': 'https://hh.ru/vacancy/32144172?query=%D0%9F%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82',
  'source': 'hh'},
 {'_id': ObjectId('5d5afddd30c104b5b6a3f881'),
  'id': 'fa7da6c3446f361991bac14a14cfcf974583feac5809afb42935fc2c80c8a151',
  'name': 'Senior java developer',
  'min_salary': 200000,
  'max_salary': 200000,
  'link': 'https://hh.ru/vacancy/32893126?query=%D0%9F%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82',
  'source': 'hh'},
 {'_id': ObjectId('5d5afddd30c104b5b6a3f888'),
  'id': '8552460f2db9963b86ecf0bf0877132fcb844fafa83fe4e5097ae497f2b2af83',
  'name': 'Backend разработчик Java (ДИТ HR)',
  'min_salary': 230000,
  'max_salary': 230000,
  'link': 'https://hh.ru/vacancy/32377870?query=%D0%9F%D1%80%D0%BE%D0%B3%

#### 4. *Модифицировать функцию, которая будет добавлять в вашу базу данных только новые вакансии с сайта

In [31]:
search_queries = ['Разработчик Python', 'Разработчик Java', 'Разработчик C++']
df_new = get_vacancies(search_queries, page_count=1)

In [32]:
df_new.head()

,id,name,min_salary,max_salary,link,source
0,4a32bb56661704ba7db0db123f16b593876fa00bbe07f0...,Разработчик Tableau,0,0,https://hh.ru/vacancy/33077842?query=%D0%A0%D0...,hh
1,ee5fbdde4daf5a96581dc923739c7f5c8342ff7dad253c...,Стажер-разработчик бекэнда (С++/Python),0,0,https://hh.ru/vacancy/31938413?query=%D0%A0%D0...,hh
2,40252ef8b03b273dc7af94117da9b0ec5b39269a61440e...,"Senior python developer в Трипстер (Python, Dj...",200000,200000,https://hh.ru/vacancy/33054998?query=%D0%A0%D0...,hh
3,483849117d342c41072aac6bf56ccbe1f1818ef2a638ca...,Стажер-разработчик (машинное обучение),0,0,https://hh.ru/vacancy/32537820?query=%D0%A0%D0...,hh
4,ceeaf3d19841cf41bcbd8974232e78b44d194e73726f22...,"Senior Front-end Developer (JavaScript, React)",200000,200000,https://hh.ru/vacancy/32144172?query=%D0%A0%D0...,hh


In [33]:
save_to_db(data_frame=df_new, db_name=db_name)

'Data saved to: vacancydb'

In [38]:
nonunique_ids = list(set(df.id) & set(df_new.id))[:10]
nonunique_ids

['c6fc2ae3867b93fe693b427137c8511ccec3b231de7a65671a365163686d2034',
 '8240e863d31e6c035959d8949d0f04052da2f8cdd769cf6b754a16b3e65259ba',
 'ceeaf3d19841cf41bcbd8974232e78b44d194e73726f222805b2424676d5ed47',
 '7212e6b42996131a93697ce0ae4ab8046207109f3ef088ad46cb4465363311ef',
 '00b6c8a1486e71178b272abff59ea9d6d20cb38787c9f7aff5616dd6773551e3',
 '4b9cdf968413ab5a6de76e06c777e8b59d434e08aecc1ed6e6c6e976eb6ff5f0',
 '5de86be0cf60c9a3f684af930a582131162357fab97070df125a93f42a9ae3f5',
 'fa7da6c3446f361991bac14a14cfcf974583feac5809afb42935fc2c80c8a151',
 '054c17814d2f1b93a9aa5a87d3d53377394160069f4054eedd02ea9c705095b3',
 '38153f547f046f0273e46c29b88dbbc33a9267b2d0098ed22fad50acfea5eaa2']

In [42]:
for nonunique_id in nonunique_ids:
    objects = db.vacancydb.find({'id': nonunique_id})
    for obj in objects:
        pprint(obj)
        print('\n')

{'_id': ObjectId('5d5afddd30c104b5b6a3f8be'),
 'id': 'c6fc2ae3867b93fe693b427137c8511ccec3b231de7a65671a365163686d2034',
 'link': 'https://www.superjob.ru/vakansii/razrabotchik-java-32047621.html',
 'max_salary': 0,
 'min_salary': 0,
 'name': 'Разработчик Java',
 'source': 'superjob'}


{'_id': ObjectId('5d5afddd30c104b5b6a3f8d7'),
 'id': '8240e863d31e6c035959d8949d0f04052da2f8cdd769cf6b754a16b3e65259ba',
 'link': 'https://www.superjob.ru/vakansii/analitik-razrabotchik-python-32360537.html',
 'max_salary': 0,
 'min_salary': 0,
 'name': 'Аналитик-разработчик Python',
 'source': 'superjob'}


{'_id': ObjectId('5d5afddd30c104b5b6a3f87c'),
 'id': 'ceeaf3d19841cf41bcbd8974232e78b44d194e73726f222805b2424676d5ed47',
 'link': 'https://hh.ru/vacancy/32144172?query=%D0%9F%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82',
 'max_salary': 200000,
 'min_salary': 200000,
 'name': 'Senior Front-end Developer (JavaScript, React)',
 'source': 'hh'}


{'_id': ObjectId('5d5afddd30c104b5b6a3f8c